In [1]:
import os
project_path = '/user/HS402/zs00774/Downloads'  # Update with your project path
os.chdir(project_path)

In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import torch
import numpy as np
import json
from transformers import Trainer, TrainingArguments,EarlyStoppingCallback
from transformers.trainer_utils import IntervalStrategy
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress FutureWarnings
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import logging
import sys
import argparse
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

from transformers import AutoImageProcessor, AutoModelForVideoClassification
import os
from transformers import TimesformerForVideoClassification

2025-08-11 00:05:51.929341: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 00:05:51.935658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754867151.944010   73919 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754867151.946529   73919 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754867151.952952   73919 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import random

class HMDBDataset(Dataset):
    def __init__(self, split_dir, clip_size=8, frame_rate=32, transform=None):
        """
        Dataset that loads from pre-split folder structure
        
        Args:
            split_dir (str): Path to split directory (train, val, or test folder)
            clip_size (int): Number of frames per clip
            frame_rate (int): Not used in random sampling, kept for compatibility
            transform: Torchvision transforms
        """
        self.split_dir = split_dir
        self.clip_size = clip_size
        self.frame_rate = frame_rate
        self.transform = transform
        
        # Get all class folders in the split directory
        self.classes = sorted([d for d in os.listdir(split_dir) 
                              if os.path.isdir(os.path.join(split_dir, d))])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        print(f"Found {len(self.classes)} classes")
        
        self.data = self._load_data()

    def _random_sample_frames(self, all_frames, num_frames):
        """Random sampling using indices"""
        num_available = len(all_frames)
        
        if num_available >= num_frames:
            # Randomly select indices without replacement
            random_indices = random.sample(range(num_available), num_frames)
            return [all_frames[i] for i in random_indices]
        else:
            # Random selection with replacement using choices
            random_indices = random.choices(range(num_available), k=num_frames)
            return [all_frames[i] for i in random_indices]
    
    def _equidistant_sample_frames(self, all_frames, num_frames):
        """Equidistant sampling using indices"""
        num_available = len(all_frames)
        
        if num_available <= num_frames:
            # Repeat frames systematically
            indices = []
            for i in range(num_frames):
                indices.append(i % num_available)
            return [all_frames[i] for i in indices]
        else:
            # Equidistant indices
            step = num_available / num_frames
            indices = [int(i * step) for i in range(num_frames)]
            return [all_frames[i] for i in indices]

    def _load_data(self):
        """Load data from the split directory"""
        data = []
        
        for class_name in self.classes:
            class_path = os.path.join(self.split_dir, class_name)
            label = self.class_to_idx[class_name]
            
            # Get all video folders in this class
            video_folders = [v for v in os.listdir(class_path) 
                           if os.path.isdir(os.path.join(class_path, v))]
            
            for video_folder in video_folders:
                video_path = os.path.join(class_path, video_folder)
                
                # Skip already processed folders
                if '_' in video_folder and video_folder.split('_')[-1].isdigit():
                    continue
                
                # Get all image frames
                all_frames = sorted([f for f in os.listdir(video_path) 
                                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
                
                if len(all_frames) == 0:
                    continue
                
                # Equidistantly sample clip_size frames (no augmentation)
                # Change this line to _random_sample_frames for random sampling
                clip_frames = self._random_sample_frames(all_frames, self.clip_size)
                
                # Add single clip to dataset
                data.append((video_path, label, 0, clip_frames))
                
        print(f"Total clips: {len(data)}")
        return data

    def __len__(self):
        return len(self.data)

    def _get_fallback_frame(self, video_path):
        """Get a fallback frame when specific frame loading fails"""
        available_frames = [f for f in os.listdir(video_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
        if available_frames:
            return Image.open(os.path.join(video_path, available_frames[0]))
        else:
            return Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))

    def _load_regular_frame(self, video_path, frame_name):
        """Load a regular frame from disk"""
        frame_path = os.path.join(video_path, frame_name)
        if os.path.exists(frame_path):
            try:
                return Image.open(frame_path)
            except Exception as e:
                return self._get_fallback_frame(video_path)
        else:
            return self._get_fallback_frame(video_path)

    def _load_frames_from_clip(self, video_path, clip_frames):
        """Load frames for a specific clip"""
        frames = []
        
        for frame_name in clip_frames:
            # Load regular frames only (no augmented frames)
            img = self._load_regular_frame(video_path, frame_name)
            frames.append(img)
        
        # Ensure exactly clip_size frames
        while len(frames) < self.clip_size:
            if frames:
                frames.append(frames[-1])  # Repeat last frame
            else:
                frames.append(Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8)))
        
        frames = frames[:self.clip_size]
        return frames

    def __getitem__(self, idx):
        video_path, label, clip_idx, clip_frames = self.data[idx]
        
        # Load frames for this specific clip
        frames = self._load_frames_from_clip(video_path, clip_frames)
        
        # Apply transforms
        if self.transform:
            frames = [self.transform(frame) for frame in frames]
        
        return torch.stack(frames), label


def get_dataloader(root_dir, batch_size=8, clip_size=8, frame_rate=32):
    """
    Create dataloaders for pre-split dataset folders
    
    Args:
        root_dir (str): Path to directory containing train, val, test folders
        batch_size (int): Batch size for dataloaders
        clip_size (int): Number of frames per clip
        frame_rate (int): Sampling rate (every Nth frame)
    
    Returns:
        tuple: (train_loader, val_loader, test_loader)
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Paths to split directories
    train_dir = os.path.join(root_dir, 'train')
    val_dir = os.path.join(root_dir, 'val')
    test_dir = os.path.join(root_dir, 'test')
    
    # Check if directories exist
    for split_dir, name in [(train_dir, 'train'), (val_dir, 'val'), (test_dir, 'test')]:
        if not os.path.exists(split_dir):
            raise ValueError(f"{name} directory not found at {split_dir}")
    
    # Create datasets for each split
    train_dataset = HMDBDataset(
        split_dir=train_dir,
        clip_size=clip_size, 
        frame_rate=frame_rate,
        transform=transform
    )
    
    val_dataset = HMDBDataset(
        split_dir=val_dir,
        clip_size=clip_size, 
        frame_rate=frame_rate,
        transform=transform
    )
    
    test_dataset = HMDBDataset(
        split_dir=test_dir,
        clip_size=clip_size, 
        frame_rate=frame_rate,
        transform=transform
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, 
        num_workers=0, pin_memory=False
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=0, pin_memory=False
    )
    
    print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}")
    return train_loader, val_loader, test_loader


In [5]:
def load_timesformer_model():
    """
    Load the pre-trained TimeSformer model for video classification.

    """
    import logging
    warnings.filterwarnings("ignore", message="Some weights of TimesformerForVideoClassification were not initialized")
    logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

    label_index_dict={'brush_hair': 0, 'cartwheel': 1, 'catch': 2, 'chew': 3, 'climb': 4, 'climb_stairs': 5, 'draw_sword': 6, 'eat': 7, 'fencing': 8, 'flic_flac': 9, 'golf': 10, 'handstand': 11, 'kiss': 12, 'pick': 13, 'pour': 14, 'pullup': 15, 'pushup': 16, 'ride_bike': 17, 'shoot_bow': 18, 'shoot_gun': 19, 'situp': 20, 'smile': 21, 'smoke': 22, 'throw': 23, 'wave': 24}
    index_label_dict={0: 'brush_hair', 1: 'cartwheel', 2: 'catch', 3: 'chew', 4: 'climb', 5: 'climb_stairs', 6: 'draw_sword', 7: 'eat', 8: 'fencing', 9: 'flic_flac',10: 'golf', 11: 'handstand', 12: 'kiss', 13: 'pick', 14: 'pour', 15: 'pullup', 16: 'pushup', 17: 'ride_bike', 18: 'shoot_bow', 19: 'shoot_gun', 20: 'situp', 21: 'smile', 22: 'smoke', 23: 'throw', 24: 'wave'}
    # Load the processor and model from Hugging Face
    processor =AutoImageProcessor.from_pretrained("facebook/timesformer-base-finetuned-k400")
    ckpt = "facebook/timesformer-base-finetuned-k400"
    model = TimesformerForVideoClassification.from_pretrained(ckpt,label2id = label_index_dict,id2label = index_label_dict,ignore_mismatched_sizes = True)

    return processor, model

In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # Top-1 accuracy
    top1_predictions = np.argmax(predictions, axis=1)
    top1_accuracy = np.mean(top1_predictions == labels)

    # Top-5 accuracy
    top5_predictions = np.argsort(predictions, axis=1)[:, -5:]
    top5_correct = np.array([labels[i] in top5_predictions[i] for i in range(len(labels))])
    top5_accuracy = np.mean(top5_correct)

    return {
        "top1_accuracy": top1_accuracy,
        "top5_accuracy": top5_accuracy,
        "eval_top1_accuracy": top1_accuracy,
        "eval_top5_accuracy": top5_accuracy,
    }


In [7]:
def train_model_with_trainer(data_dir, epochs, batch_size, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    train_loader, val_loader, test_loader = get_dataloader(data_dir, batch_size)
    processor, model = load_timesformer_model()

    train_dataset = train_loader.dataset
    val_dataset = val_loader.dataset

    def data_collator(batch):
        videos = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        return {
            'pixel_values': videos,
            'labels': labels
        }
    optimizer = optim.SGD(model.parameters(), momentum= 0.9, weight_decay= 1e-3,
    			  lr= learning_rate)

    training_args = TrainingArguments(output_dir = "/user/HS402/zs00774/Downloads/results",
                                      overwrite_output_dir = True,
                                      eval_strategy = 'epoch',
                                      per_device_train_batch_size = 8,
                                      per_device_eval_batch_size = 8,
                                      num_train_epochs = epochs,
                                      logging_dir = os.path.normpath(os.path.join('/user/HS402/zs00774/Downloads/results', 'logs')),
                                      logging_strategy = "epoch",
                                      save_strategy = 'epoch',
                                      save_total_limit = 1,
                                      remove_unused_columns = False,
                                      load_best_model_at_end = True,
                                      metric_for_best_model = 'eval_top1_accuracy',
                                      greater_is_better = True,
                                      report_to = "tensorboard",
                                      push_to_hub = False,
                                     save_only_model = True)

    early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 4 )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        optimizers=(optimizer,None),
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks = [early_stopping_callback])

    print("Starting training...")
    trainer.train()

    print("\nEvaluating on test set...")
    test_dataset = test_loader.dataset
    test_results = trainer.evaluate(eval_dataset=test_dataset)

    print(f"\n=== FINAL TEST RESULTS ===")
    print(f"Test Top-1 Accuracy: {test_results.get('eval_top1_accuracy', 'N/A'):.4f}")
    print(f"Test Top-5 Accuracy: {test_results.get('eval_top5_accuracy', 'N/A'):.4f}")
    print(f"Test Loss: {test_results.get('eval_loss', 'N/A'):.4f}")

    trainer.save_model("/user/HS402/zs00774/Downloads/timesformer_model")

    with open("/user/HS402/zs00774/Downloads/test_results.json", "w") as f:
        json.dump(test_results, f, indent=2)

    print("Training complete. Model and results saved.")
    return test_results


In [8]:
def evaluate_saved_model(model_path, data_dir, batch_size=8):
    from transformers import TimesformerForVideoClassification

    model = TimesformerForVideoClassification.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    _, _, test_loader = get_dataloader(data_dir, batch_size)

    def data_collator(batch):
        videos = torch.stack([item[0] for item in batch])
        labels = torch.tensor([item[1] for item in batch])
        return {
            'pixel_values': videos,
            'labels': labels
        }

    trainer = Trainer(
        model=model,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    results = trainer.evaluate(eval_dataset=test_loader.dataset)

    print(f"=== Model Evaluation: {model_path} ===")
    print(f"Top-1 Accuracy: {results.get('eval_top1_accuracy', 'N/A'):.4f}")
    print(f"Top-5 Accuracy: {results.get('eval_top5_accuracy', 'N/A'):.4f}")
    print(f"Loss: {results.get('eval_loss', 'N/A'):.4f}")

    return results

In [11]:
train_model_with_trainer(
        data_dir='/user/HS402/zs00774/Downloads/dataset',
        epochs=8,
        batch_size=8,
        learning_rate=0.0035
    )

Using device: cuda
Found 25 classes
Total clips: 900
Found 25 classes
Total clips: 225
Found 25 classes
Total clips: 125
Train: 900 | Val: 225 | Test: 125
Starting training...


Epoch,Training Loss,Validation Loss,Top1 Accuracy,Top5 Accuracy
1,2.280100,1.292632,0.777778,0.955556
2,0.876100,0.720565,0.853333,0.973333
3,0.459600,0.553887,0.871111,0.982222
4,0.289300,0.469944,0.897778,0.982222
5,0.195300,0.423090,0.884444,0.982222
6,0.142000,0.402731,0.893333,0.986667
7,0.109700,0.391436,0.906667,0.986667
8,0.093900,0.388123,0.906667,0.986667



Evaluating on test set...



=== FINAL TEST RESULTS ===
Test Top-1 Accuracy: 0.9120
Test Top-5 Accuracy: 1.0000
Test Loss: 0.2922
Training complete. Model and results saved.


{'eval_top1_accuracy': 0.912,
 'eval_top5_accuracy': 1.0,
 'eval_loss': 0.29224082827568054,
 'eval_runtime': 11.5371,
 'eval_samples_per_second': 10.835,
 'eval_steps_per_second': 1.387,
 'epoch': 8.0}